In [16]:
import pandas as pd
import collections
import torch
import numpy as np

df_train1 = pd.read_csv('BIKER_train.QApair.csv')
df_train1=df_train1[["title","answer"]]

# df_train2 = pd.read_csv('Big_train.QApair.csv')
# df_train2=df_train2[["title","answer"]]

df_train3 = pd.read_csv('BIKER_querys_final.csv')
df_train3=df_train3[["title","answer"]]

# df_test = pd.read_csv("../BERT_sim/BIKER_1k_rdm.csv")
# df_test = df_test[["title","answer"]]

# df_sim_test = pd.read_csv('../BERT_sim/Biker_1k_similar.csv')
# df_sim_test = df_sim_test[["title","answer"]]


In [17]:
df_train = [df_train1,df_train3]
df_train = pd.concat(df_train)

In [18]:
df_train = df_train.drop_duplicates(subset=['title'], keep='first')

In [19]:
df_multi=df_train[df_train["answer"].str.contains(",")]

In [20]:
len(df_multi)

10164

In [21]:
len(list(set(df_train["answer"].to_list())))

10755

In [41]:
import random
from random import sample
random.seed(1)


def generate_random_sampling_min_m_max_n(df,m=None,p=None,n=None):
    Passage_dict = {}
    Corpus_dict = {}
    Answers_dict = {}
    Triplets= []
    api_list=[]
    
    gps = df.groupby("answer")

    for k,v in gps:
        if len(v)>=m:
            api_list.append(k)
    df = df[df["answer"].isin(api_list)]
    df = df.reset_index(drop=True)
    
    print("len train",len(df_train))
    
    gps = df.groupby("answer")
    
    title_list = df["title"].to_list()
    answer_list = df["answer"].to_list()
    
    print("len title_list",len(title_list))
    print("len answer_list",len(answer_list))
    
    for idx,t in enumerate(title_list):
        Corpus_dict[idx] = t
        
    for idx,t in enumerate(answer_list):
        Answers_dict[idx] = t
        
    counter=0
    
    for idx in range(len(title_list)):
        counter+=1
        if counter %10000 == 0: print(counter)
        label = Answers_dict[idx]
        
        gp = gps.get_group(label)
        
        same_api_idx_gp = list(gp.index)
        diff_api_idx_gp = list(set(range(len(answer_list)))-set(same_api_idx_gp))

        if len(same_api_idx_gp)>p:
            same_api_idx_gp=sample(same_api_idx_gp,p)
            
        diff_api_idx_gp=sample(diff_api_idx_gp,n)

        Passage_dict[idx] = [same_api_idx_gp,diff_api_idx_gp]
        
    print("len Corpus_dict",len(Corpus_dict))
    print("len Answers_dict",len(Answers_dict))
    print("len Passage_dict",len(Passage_dict))
    return df,Corpus_dict,Answers_dict,Passage_dict
    
def get_triplets(Passage_dict):
    Triplets= []
    for k, v in Passage_dict.items():
        for x in v[0]:
            for y in v[1]:
                Triplets.append([k,x,y])
    return Triplets

def get_rel_doc(df,evaluate_queries, Answers_dict):
    rel_dict ={}
    gps = df.groupby("answer")
    
    for q in evaluate_queries:
        label = Answers_dict[q]
        
        gp = gps.get_group(label)
        
        same_api_idx_gp = list(gp.index)
#         print(same_api_idx_gp)
#         print(q)
        same_api_idx_gp = list(set(same_api_idx_gp)-set([q]))

        rel_dict[q] = [same_api_idx_gp]
    return rel_dict

In [42]:
p=10
n=10
df_train_filtered,Corpus_dict,Answers_dict,Passage_dict = generate_random_sampling_min_m_max_n(df_train,m=5,p=p,n=n)

len train 34030
len title_list 21479
len answer_list 21479
10000
20000
len Corpus_dict 21479
len Answers_dict 21479
len Passage_dict 21479


In [45]:
evaluate_queries = sample(list(Passage_dict.keys()),1000)
print("evaluate_queries",len(evaluate_queries))
evaluate_Corpus = list(set(range(len(Answers_dict)))-set(evaluate_queries))
evaluate_rel_doc = get_rel_doc(df_train_filtered,evaluate_queries,Answers_dict)

folder = "random_dis_query"
if not os.path.exists(folder):
    os.mkdir(folder) 
data_foler = folder+"/"

for i in range(10):
    evaluate_queries = sample(list(Passage_dict.keys()),1000)
    print("evaluate_queries",len(evaluate_queries))
    evaluate_Corpus = list(set(range(len(Answers_dict)))-set(evaluate_queries))
    evaluate_rel_doc = get_rel_doc(df_train_filtered,evaluate_queries,Answers_dict)
    with open(data_foler+'evaluate_queries'+str(i)+'.json', 'w') as jsonfile:
        json.dump(evaluate_queries, jsonfile)

    with open(data_foler+'evaluate_Corpus'+str(i)+'.json', 'w') as jsonfile:
        json.dump(evaluate_Corpus, jsonfile)


evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000


In [24]:
p=10
n=10
df_train_filtered_multi,Corpus_dict_multi,Answers_dict_multi,Passage_dict_multi = generate_random_sampling_min_m_max_n(df_multi,m=5,p=p,n=n)

len train 34030
len title_list 1991
len answer_list 1991
len Corpus_dict 1991
len Answers_dict 1991
len Passage_dict 1991


In [46]:
evaluate_multi_queries = sample(list(Passage_dict_multi.keys()),1000)
print("evaluate_queries",len(evaluate_multi_queries))
evaluate_multi_Corpus = list(set(range(len(Answers_dict_multi)))-set(evaluate_multi_queries))
evaluate_multi_rel_doc = get_rel_doc(df_train_filtered_multi,evaluate_multi_queries,Answers_dict_multi)

folder = "random_dis_query_multi"
if not os.path.exists(folder):
    os.mkdir(folder) 
data_foler = folder+"/"

for i in range(10):
    evaluate_multi_queries = sample(list(Passage_dict_multi.keys()),1000)
    print("evaluate_queries",len(evaluate_multi_queries))
    evaluate_multi_Corpus = list(set(range(len(Answers_dict_multi)))-set(evaluate_multi_queries))
    evaluate_multi_rel_doc = get_rel_doc(df_train_filtered_multi,evaluate_multi_queries,Answers_dict_multi)
    
    with open(data_foler+'evaluate_queries'+str(i)+'.json', 'w') as jsonfile:
        json.dump(evaluate_multi_queries, jsonfile)

    with open(data_foler+'evaluate_Corpus'+str(i)+'.json', 'w') as jsonfile:
        json.dump(evaluate_multi_Corpus, jsonfile)


evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000
evaluate_queries 1000


In [26]:
filtered_biker_test = []
for index, row in df_train3.iterrows():
    y = str(list(eval(row['answer'])))
    if y in Answers_dict.values():
        filtered_biker_test.append((row['title'],y))
len(filtered_biker_test)
df_filtered_biker_test = pd.DataFrame(data = filtered_biker_test,columns=["title","answer"])
df_filtered_biker_test.to_csv("Biker_test_filtered.csv")

In [32]:
import json
import os



# for p in [1,3,5,10,15]:
#     for n in [1,3,5,10,15]:
        
for i in range(10):
    df_train1= df_train.sample(n=3000)
    p=10
    n=10

    df_train_filtered,Corpus_dict,Answers_dict,Passage_dict = generate_random_sampling_min_m_max_n(df_train1,m=5,p=p,n=n)

    evaluate_queries = sample(list(Passage_dict.keys()),250)
    print("evaluate_queries",len(evaluate_queries))
    evaluate_Corpus = list(set(range(len(Answers_dict)))-set(evaluate_queries))
    evaluate_rel_doc = get_rel_doc(df_train_filtered,evaluate_queries,Answers_dict)

    test_queries = sample(list(Passage_dict.keys()),250)
    test_Corpus = list(set(range(len(Answers_dict)))-set(evaluate_queries))
    test_rel_doc = get_rel_doc(df_train_filtered,evaluate_queries,Answers_dict)

    triplets = get_triplets(Passage_dict)
    print("triplets",len(triplets))

    gps = df_train_filtered.groupby("answer")
    print("gps",len(gps))

    
    folder = "random_dis_3k_min_5_max_"+str(p)+"_ir_"+str(n)+"_"+str(i)
    if not os.path.exists(folder):
        os.mkdir(folder) 
    data_foler = folder+"/"
    with open(data_foler+'Corpus_dict.json', 'w') as jsonfile:
        json.dump(Corpus_dict, jsonfile)

    with open(data_foler+'Answers_dict.json', 'w') as jsonfile:
        json.dump(Answers_dict, jsonfile)

    with open(data_foler+'Passage_dict.json', 'w') as jsonfile:
        json.dump(Passage_dict, jsonfile)


    with open(data_foler+'evaluate_queries.json', 'w') as jsonfile:
        json.dump(evaluate_queries, jsonfile)

    with open(data_foler+'evaluate_Corpus.json', 'w') as jsonfile:
        json.dump(evaluate_Corpus, jsonfile)

    with open(data_foler+'evaluate_rel_doc.json', 'w') as jsonfile:
        json.dump(evaluate_rel_doc, jsonfile)


    with open(data_foler+'test_queries.json', 'w') as jsonfile:
        json.dump(test_queries, jsonfile)

    with open(data_foler+'test_Corpus.json', 'w') as jsonfile:
        json.dump(test_Corpus, jsonfile)

    with open(data_foler+'test_rel_doc.json', 'w') as jsonfile:
        json.dump(test_rel_doc, jsonfile)

len train 34030
len title_list 1046
len answer_list 1046
len Corpus_dict 1046
len Answers_dict 1046
len Passage_dict 1046
evaluate_queries 250
triplets 93360
gps 85
len train 34030
len title_list 1007
len answer_list 1007
len Corpus_dict 1007
len Answers_dict 1007
len Passage_dict 1007
evaluate_queries 250
triplets 88090
gps 90
len train 34030
len title_list 957
len answer_list 957
len Corpus_dict 957
len Answers_dict 957
len Passage_dict 957
evaluate_queries 250
triplets 82950
gps 87
len train 34030
len title_list 1083
len answer_list 1083
len Corpus_dict 1083
len Answers_dict 1083
len Passage_dict 1083
evaluate_queries 250
triplets 96410
gps 90
len train 34030
len title_list 1060
len answer_list 1060
len Corpus_dict 1060
len Answers_dict 1060
len Passage_dict 1060
evaluate_queries 250
triplets 92550
gps 93
len train 34030
len title_list 1087
len answer_list 1087
len Corpus_dict 1087
len Answers_dict 1087
len Passage_dict 1087
evaluate_queries 250
triplets 93160
gps 98
len train 34030

In [38]:
# evaluate_multi_queries = sample(list(Passage_dict_multi.keys()),1000)
# print("evaluate_queries",len(evaluate_multi_queries))
# evaluate_multi_Corpus = list(set(range(len(Answers_dict_multi)))-set(evaluate_multi_queries))
# evaluate_multi_rel_doc = get_rel_doc(df_train_filtered_multi,evaluate_multi_queries,Answers_dict_multi)

data_folder = '../@SentenceBert_finetune/full_data_min_5_max_10_ir_10/'
with open(data_folder+'evaluate_multi_queries.json', 'w') as jsonfile:
    json.dump(evaluate_multi_queries, jsonfile)

with open(data_folder+'evaluate_multi_Corpus.json', 'w') as jsonfile:
    json.dump(evaluate_multi_Corpus, jsonfile)

with open(data_folder+'evaluate_multi_rel_doc.json', 'w') as jsonfile:
    json.dump(evaluate_multi_rel_doc, jsonfile)
    


test_corpus = []
test_answers = []
collection_filepath = os.path.join(data_folder, 'evaluate_multi_Corpus.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
    data = json.load(fIn)
    for k in data:
        test_corpus.append(corpus[k])
        test_answers.append(Answers[k])

test_queries = []
test_queries_answers = []
collection_filepath = os.path.join(data_folder, 'evaluate_multi_queries.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
    data = json.load(fIn)
    for k in data:
        test_queries.append(corpus[k])
        test_queries_answers.append(Answers[k])
        
        
df1 = pd.DataFrame(data = list(zip(test_corpus,test_answers)),columns=["title","answer"])
df1.to_csv("test_corpus_multi_min_5_max_10_ir_10.csv")

df2 = pd.DataFrame(data = list(zip(test_queries,test_queries_answers)),columns=["title","answer"])
df2.to_csv("test_queries_multi_min_5_max_10_ir_10.csv")

In [1]:
### Now we read the MS Marco dataset
import os
data_folder = '../@SentenceBert_finetune/full_data_min_5_max_10_ir_10/'
os.makedirs(data_folder, exist_ok=True)
import json 
import pandas as pd

# in the order of 1 to 30k
corpus = []
collection_filepath = os.path.join(data_folder, 'Corpus_dict.json')
#"evaluate_Corpus_min_2_max_10.json"
with open(collection_filepath, 'r', encoding='utf8') as fIn:
    data = json.load(fIn)
    for k in range(len(data)):
        corpus.append(data[str(k)])

# in the order of 1 to 30k
Answers = []
collection_filepath = os.path.join(data_folder, 'Answers_dict.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
    data = json.load(fIn)
    for k in range(len(data)):
        Answers.append(data[str(k)])


evaluate_corpus = []
evaluate_answers = []
collection_filepath = os.path.join(data_folder, 'evaluate_Corpus.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
    data = json.load(fIn)
    for k in data:
        evaluate_corpus.append(corpus[k])
        evaluate_answers.append(Answers[k])

queries = []
queries_answers = []
collection_filepath = os.path.join(data_folder, 'evaluate_queries.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
    data = json.load(fIn)
    for k in data:
        queries.append(corpus[k])
        queries_answers.append(Answers[k])
        
test_corpus = []
test_answers = []
collection_filepath = os.path.join(data_folder, 'test_Corpus.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
    data = json.load(fIn)
    for k in data:
        test_corpus.append(corpus[k])
        test_answers.append(Answers[k])

test_queries = []
test_queries_answers = []
collection_filepath = os.path.join(data_folder, 'test_queries.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
    data = json.load(fIn)
    for k in data:
        test_queries.append(corpus[k])
        test_queries_answers.append(Answers[k])

In [2]:
import pandas as pd
df_so_test = pd.read_csv("SO_test_latest.csv")

In [3]:
df_so_test

,title,answer
0,How can I resize an image from a URL into a JP...,['javax.swing.ImageIcon.getImage']
1,array with random ints but no duplicates and f...,"['java.util.Random.nextInt', 'java.util.stream..."
2,How to execute multiple SQL queries defined in...,"['java.lang.String.split', 'java.nio.file.File..."
3,StructField issue at compilation when specifyi...,['java.util.Arrays.asList']
4,Jsoup combine multiple Evaluator,['java.lang.String.join']
...,...,...
594,How to conver date on arguments into format da...,"['java.text.SimpleDateFormat.parse', 'java.tex..."
595,"""BufferedReader"" vs. ""java.io.BufferedReader""....",['java.io.BufferedReader.readLine']
596,"How to hide the warning ""This type of file can...",['java.lang.System.setProperty']
597,Some way to get files CRC from file system?,"['java.util.zip.Checksum.update', 'java.util.z..."


In [6]:
filtered_so_test = []
for index, row in df_so_test.iterrows():
    y = str(list(eval(row['answer'])))
    if y in Answers:
        filtered_so_test.append((row['title'],y))
len(filtered_so_test)
filtered_so_test = pd.DataFrame(data = filtered_so_test,columns=["title","answer"])
# filtered_so_test.to_csv("Biker_test_filtered.csv")

In [7]:
filtered_so_test

,title,answer
0,StructField issue at compilation when specifyi...,['java.util.Arrays.asList']
1,Jsoup combine multiple Evaluator,['java.lang.String.join']
2,Update class variables on button click,['java.lang.Integer.valueOf']
3,Changing colour of all buttons in netbeans,['java.lang.String.valueOf']
4,SQLite database update doesn't update,['java.lang.String.valueOf']
...,...,...
222,Java - Converting character of string to upper...,['java.util.regex.Pattern.compile']
223,Will methods in lambda only be evaluated when ...,['java.lang.Runnable.run']
224,How do i fix a non static method being called ...,['java.util.Scanner.next']
225,"""BufferedReader"" vs. ""java.io.BufferedReader""....",['java.io.BufferedReader.readLine']


In [9]:
filtered_so_test.to_csv("SO_test_filtered.csv")

In [12]:
print(len(corpus))
print(len(evaluate_corpus))

21479
20479


In [33]:
len(list(set(Answers)))

955

In [9]:
counter=0
list_=[]
for l in Answers:
    if "," in l:
        list_.append(l)
        counter +=1
print(counter)

len(list(set(list_)))

1991


185

In [34]:
df1 = pd.DataFrame(data = list(zip(test_corpus,test_answers)),columns=["title","answer"])
df1.to_csv("test_corpus_min_5_max_10_ir_10.csv")

df2 = pd.DataFrame(data = list(zip(test_queries,test_queries_answers)),columns=["title","answer"])
df2.to_csv("test_queries_min_5_max_10_ir_10.csv")